In [6]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [7]:
with open('intents.json') as file:
    data = json.load(file)

In [8]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['hello', 'hi', 'hey'],
   'responses': ['Hello! How can I assist you today?',
    'Hi there! How can I help you?']},
  {'tag': 'farewell',
   'patterns': ['goodbye', 'bye', 'see you later'],
   'responses': ['Goodbye! Have a great day!', 'See you later! Take care.']},
  {'tag': 'job',
   'patterns': ['what can you do?',
    'what all can you perform?',
    'what are the options?',
    'what can the options you have?'],
   'responses': ['I can do Arthemetic Operations and can predict temperature']},
  {'tag': 'Game',
   'patterns': ['I would like to paly',
    'What game can I play now?',
    'What is the condition outside?',
    'best conditions to play'],
   'responses': ['Please Enter your Location and choose from the options']},
  {'tag': 'add',
   'patterns': ['Add', 'Addition', 'Add numbers'],
   'responses': ['Sure, what are the numbers you want to add?']},
  {'tag': 'subtract',
   'patterns': ['Subtract', 'Subtraction', 'Subtract 

In [9]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [10]:
training_labels

['greeting',
 'greeting',
 'greeting',
 'farewell',
 'farewell',
 'farewell',
 'job',
 'job',
 'job',
 'job',
 'Game',
 'Game',
 'Game',
 'Game',
 'add',
 'add',
 'add',
 'subtract',
 'subtract',
 'subtract',
 'multiply',
 'multiply',
 'multiply',
 'divison',
 'divison']

In [11]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)


In [12]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
Total params: 16,680
Trainable params: 16,680
Non-trainable params: 0
____________________________________________________

In [15]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
1/1 [==============================] - 1s 1s/step - loss: 2.0791 - accuracy: 0.1600
Epoch 2/550
1/1 [==============================] - 0s 11ms/step - loss: 2.0786 - accuracy: 0.1600
Epoch 3/550
1/1 [==============================] - 0s 12ms/step - loss: 2.0781 - accuracy: 0.1600
Epoch 4/550
1/1 [==============================] - 0s 14ms/step - loss: 2.0776 - accuracy: 0.1600
Epoch 5/550
1/1 [==============================] - 0s 8ms/step - loss: 2.0771 - accuracy: 0.1600
Epoch 6/550
1/1 [==============================] - 0s 13ms/step - loss: 2.0767 - accuracy: 0.1600
Epoch 7/550
1/1 [==============================] - 0s 8ms/step - loss: 2.0762 - accuracy: 0.1600
Epoch 8/550
1/1 [==============================] - 0s 9ms/step - loss: 2.0758 - accuracy: 0.1600
Epoch 9/550
1/1 [==============================] - 0s 9ms/step - loss: 2.0753 - accuracy: 0.1600
Epoch 10/550
1/1 [==============================] - 0s 12ms/step - loss: 2.0748 - accuracy: 0.1600
Epoch 11/550
1/1 [=======

In [16]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import re
import colorama 
colorama.init()
from colorama import Fore, Style, Back
import requests
import random
import pickle
with open("intents.json") as file:
    data = json.load(file)
def chat():
    # load trained model
    model = keras.models.load_model('chat_model')
    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)
    # parameters
    max_len = 20
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break
        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
        print(tag)
        for i in data['intents']:
          if tag == 'add':
              inp = input()
              numbers_pattern = re.compile(r'\d+')
              numbers = numbers_pattern.findall(inp)
              num1, num2 = float(numbers[0]), float(numbers[1])
              result = num1 + num2
              print(f"Bot: The sum of {num1} and {num2} is {result}")
              break
          elif tag == 'subtract':
              inp = input()
              numbers_pattern = re.compile(r'\d+')
              numbers = numbers_pattern.findall(inp)
              num1, num2 = float(numbers[0]), float(numbers[1])
              result = num1 - num2
              print(f"Bot: The difference between {num1} and {num2} is {result}")
              break
          elif tag == 'multiply':
              inp = input()
              numbers_pattern = re.compile(r'\d+')
              numbers = numbers_pattern.findall(inp)
              num1, num2 = float(numbers[0]), float(numbers[1])
              result = num1 * num2
              print(f"Bot: The product of {num1} and {num2} is {result}")
              break
          elif tag == 'division':
              inp = input()
              numbers_pattern = re.compile(r'\d+')
              numbers = numbers_pattern.findall(inp)
              num1, num2 = float(numbers[0]), float(numbers[1])
              if num2 != 0:
                result = num1/num2
                print(f"The Division is {result}")
                break
              else:
                print(f"The division is infinity")
                break
          elif tag == 'Game':
              API_KEY = "06f070197b1f60e55231f8c46658d077"
              CITY_NAME = input("CITY_NAME : ")
              print("Please select an option:")
              print("1. Cricket")
              print("2. Ice Hockey")
              print("3. American Football")
              selected_option = int(input("Enter your selection: "))
              url = f"https://api.openweathermap.org/data/2.5/weather?q={CITY_NAME}&appid={API_KEY}&units=metric"
              print(url)
              response = requests.get(url)
              #print(response)
              if response.status_code == 200:
                  print(True)
                  # Parse the JSON data from the response
                  dat = response.json()
                  print(dat)
                  # Extract the relevant weather data
                  temperature = dat['main']['temp']
                  feels_like = dat['main']['feels_like']
                  description = dat['weather'][0]['description']
                  min = dat['main']['temp_min']
                  max = dat['main']['temp_max']
                  if selected_option == 1:
                    print(True)
                    if min >=18 and max <=32:

                      print("Yes, It is the best condition to play")
                      break
                    else :
                      print("The weather condition is not suitable to play")
                      break
                  elif selected_option == 2:
                    if min>=-5 and max<=-2:
                      print("Yes, It is the best condition to play")
                      break
                    else :
                      print("The weather condition is not suitable to play")
                      break
                  elif selected_option == 3:
                    if min >=10 and max<=26:
                      print("Yes, It is the best condition to play")
                      break
                    else :
                      print("The weather condition is not suitable to play")
                      break
              else:
                  # Print an error message if the response was not successful
                  print(f"Error getting weather data: {response.status_code}")

          if i['tag'] == tag and tag not in ['addition','subtraction','multiplication','division']:
              print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()


Start messaging with the bot (type quit to stop)!
User: i would like to play
1/1 [==============================] - 0s 75ms/step
['Game']
CITY_NAME : Hyderabad
Please select an option:
1. Cricket
2. Ice Hockey
3. American Football
Enter your selection: 1
https://api.openweathermap.org/data/2.5/weather?q=Hyderabad&appid=06f070197b1f60e55231f8c46658d077&units=metric
True
{'coord': {'lon': 78.4744, 'lat': 17.3753}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 29.23, 'feels_like': 27.85, 'temp_min': 28.73, 'temp_max': 29.23, 'pressure': 1017, 'humidity': 26}, 'visibility': 6000, 'wind': {'speed': 4.12, 'deg': 90}, 'clouds': {'all': 40}, 'dt': 1678281605, 'sys': {'type': 1, 'id': 9214, 'country': 'IN', 'sunrise': 1678237205, 'sunset': 1678280057}, 'timezone': 19800, 'id': 1269843, 'name': 'Hyderabad', 'cod': 200}
True
Yes, It is the best condition to play
User: Hello
1/1 [==============================] - 

In [30]:
import requests

# Replace YOUR_API_KEY with your actual API key
API_KEY = "06f070197b1f60e55231f8c46658d077"

# Replace CITY_NAME with the name of the city you want to get weather data for
CITY_NAME = input("CITY_NAME : ")

# Define the API endpoint URL
url = f"https://api.openweathermap.org/data/2.5/weather?q={CITY_NAME}&appid={API_KEY}&units=metric"

# Send a GET request to the API endpoint and get the response
response = requests.get(url)

# Check if the response was successful (HTTP status code 200)
if response.status_code == 200:
    # Parse the JSON data from the response
    data = response.json()
    print(data)
    # Extract the relevant weather data
    temperature = data['main']['temp']
    feels_like = data['main']['feels_like']
    description = data['weather'][0]['description']
    
    # Print the weather data
    print(f"Temperature: {temperature}°C")
    print(f"Feels like: {feels_like}°C")
    print(f"Description: {description}")
else:
    # Print an error message if the response was not successful
    print(f"Error getting weather data: {response.status_code}")


CITY_NAME : hyderabad
{'coord': {'lon': 78.4744, 'lat': 17.3753}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03n'}], 'base': 'stations', 'main': {'temp': 30.23, 'feels_like': 28.64, 'temp_min': 29.73, 'temp_max': 30.23, 'pressure': 1016, 'humidity': 25}, 'visibility': 6000, 'wind': {'speed': 4.12, 'deg': 80}, 'clouds': {'all': 40}, 'dt': 1678280889, 'sys': {'type': 1, 'id': 9214, 'country': 'IN', 'sunrise': 1678237205, 'sunset': 1678280057}, 'timezone': 19800, 'id': 1269843, 'name': 'Hyderabad', 'cod': 200}
Temperature: 30.23°C
Feels like: 28.64°C
Description: scattered clouds
